In [1]:
import multiprocessing
#finding the number of cores
print(multiprocessing.cpu_count())

16


In [5]:
import psutil
#Total RAM
print(f"Total RAM installed: {round(psutil.virtual_memory().total/1000000000, 2)} GB")
#Available RAM
print(f"Available RAM: {round(psutil.virtual_memory().available/1000000000, 2)} GB")
#Used RAM
print(f"Used RAM: {round(psutil.virtual_memory().used/1000000000, 2)} GB")
#RAM usage
print(f"RAM usage: {psutil.virtual_memory().percent} %")

Total RAM installed: 17.04 GB
Available RAM: 2.46 GB
Used RAM: 14.58 GB
RAM usage: 85.6 %


In [15]:
# Creating a local cluster

from dask.distributed import Client
client = Client()

In [16]:
client

Client Scheduler: tcp://127.0.0.1:51466 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 15.87 GiB


In [8]:
#Loading the data
import dask.dataframe as dd


data_ks_2018 = dd.read_csv("C:\\Users\\Bismark\\Desktop\\Mike_Ansah_assignments\\Sec_Sem\\Machine_learning\\ks_2018_data_encoded.csv")

In [9]:
#Checking number of partitions

print(data_ks_2018.npartitions)

1


In [10]:
#Repartitioning data for easier distribution on cluster

data_ks_2018 = data_ks_2018.repartition(npartitions=10)

print(data_ks_2018.npartitions)

10


In [12]:
#Viewing sample of the data

data_ks_2018.head()

,main_category_1,main_category_2,main_category_3,main_category_4,main_category_5,main_category_6,main_category_7,main_category_8,main_category_9,main_category_10,...,usd_pledged,usd_goal,launch_gap,deadline_year,deadline_month,deadline_day,launched_year,launched_month,launched_day,state_target
0,1,0,0,0,0,0,0,0,0,0,...,0.0,1533.95,59,2015,10,9,2015,8,11,1
1,0,1,0,0,0,0,0,0,0,0,...,1.0,5000.00,30,2012,4,16,2012,3,17,1
2,0,0,1,0,0,0,0,0,0,0,...,1283.0,19500.00,56,2015,8,29,2015,7,4,0
3,0,0,0,1,0,0,0,0,0,0,...,1205.0,1000.00,20,2014,12,21,2014,12,1,2
4,1,0,0,0,0,0,0,0,0,0,...,0.0,2406.39,30,2013,10,9,2013,9,9,1


In [17]:
#Taking target of main data
data_X = data_ks_2018.drop('state_target', axis =1)

data_y= data_ks_2018.state_target

In [24]:
#Splitting the data
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.3, random_state=42)

In [78]:
#Balancing the training data
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
#X_train_resam, y_train_resam = oversample.fit_resample(scaled_X_train, y_train)
X_train_resam, y_train_resam = oversample.fit_resample(X_train, y_train)

In [47]:
X_train_resam.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,2.693599,-0.424100,-0.449167,-0.250258,-0.176845,-0.249538,-0.16838,-0.267082,-0.18914,-0.317326,...,-0.800149,-0.576340,-0.711545,1.910303,0.424327,0.993333,-0.696969,0.455399,0.484723,-0.493488
1,-0.371251,2.357936,-0.449167,-0.250258,-0.176845,-0.249538,-0.16838,-0.267082,-0.18914,-0.317326,...,-0.755002,-0.575894,-0.049416,-0.272733,-1.100848,-0.819008,0.075569,-1.061886,-1.024951,0.187442
2,-0.371251,-0.424100,2.226341,-0.250258,-0.176845,-0.249538,-0.16838,-0.267082,-0.18914,-0.317326,...,-0.168090,-0.004216,2.720559,1.684472,0.424327,0.389219,1.510282,0.455399,0.182789,-1.287907
3,-0.371251,-0.424100,-0.449167,3.995881,-0.176845,-0.249538,-0.16838,-0.267082,-0.18914,-0.317326,...,-0.077795,-0.038998,-0.813547,-1.025504,-0.084064,1.597447,0.627382,-0.050363,1.692463,-1.628372
4,-0.371251,-0.424100,-0.449167,3.995881,-0.176845,-0.249538,-0.16838,-0.267082,-0.18914,-0.317326,...,1.005736,0.217855,-0.049416,-0.272733,-1.100848,-0.214894,-1.469507,-1.061886,-0.421081,-1.401395


In [57]:
X_train_resam.shape

(416816, 39)

In [58]:
y_train_resam.shape

(416816,)

In [69]:
X_test.shape

(Delayed('int-b06e6738-1882-400e-9da6-bb859023def3'), 39)

In [65]:
y_test

Dask Series Structure:
npartitions=10
    int64
      ...
    ...  
      ...
      ...
Name: state_target, dtype: int64
Dask Name: split, 42 tasks

In [89]:
# Modeling a Naive Bayes classification model using the cluster
%%time 

from sklearn.naive_bayes import BernoulliNB
from dask_ml.wrappers import Incremental
from dask.distributed import Client
import joblib

with joblib.parallel_backend('dask'):
    nb = BernoulliNB()
    parallel_nb = Incremental(nb)

    parallel_nb.fit(X_train_resam, y_train_resam, classes=[0,1])

parallel_nb.score(X_test, y_test)

Wall time: 947 ms


0.5206041221515879